## Event Synchronization (ES) networks 

Event Synchronization (ES) is a statistical method used to quantify the synchrony between event time series by measuring the relative timing of events in two time series. ES has become particularly useful in climate science for analyzing extreme events across different geographical locations.

This notebook demonstrates how to use the `dominosee` package to perform Event Synchronization for the temporal positions of events and construct event-based climate networks from time series data.

### Input Data

We'll start by creating a synthetic dataset to demonstrate the ES workflow. For real-world applications, you would typically load climate data from NetCDF files or other sources. In this example, we'll generate a simple dataset with the Standardized Precipitation Index (SPI) values, which are commonly used to identify drought conditions.

We'll create a synthetic dataset with the following properties:
- Spatial dimensions: 20x20 grid points (latitude x longitude)
- Temporal dimension: 365 days (daily data for one year)
- Variable: SPI1

In [2]:
import numpy as np
import xarray as xr

xr.set_options(display_expand_data=False)

In [3]:
# Using smaller dimensions for demonstration
nx_demo, ny_demo, nt_demo = 20, 20, 365

# Create coordinates
lats = np.linspace(-90, 90, nx_demo)
lons = np.linspace(-180, 180, ny_demo)
times = xr.date_range("1950-01-01", periods=nt_demo, freq="D")

# Create standard normal data for SPI values (mean=0, std=1)
spi_data = np.random.normal(0, 1, size=(nx_demo, ny_demo, nt_demo))  # Standard normal distribution

# Create xarray Dataset
spi = xr.Dataset(
    data_vars={
        "SPI1": (["lat", "lon", "time"], spi_data)
    },
    coords={
        "lat": lats,
        "lon": lons,
        "time": times
    }
)

spi

<xarray.Dataset> Size: 1MB
Dimensions:  (lat: 20, lon: 20, time: 365)
Coordinates:
  * lat      (lat) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lon      (lon) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * time     (time) datetime64[ns] 3kB 1950-01-01 1950-01-02 ... 1950-12-31
Data variables:
    SPI1     (lat, lon, time) float64 1MB -0.6445 -0.3062 ... 0.3327 -0.5354

### Extract the timing of extreme events

Event Synchronization analysis requires the extraction of discrete event positions from continuous time series data. In this step, we convert our continuous SPI time series into binary event time series by applying a threshold.

We'll use the `get_event` function from the `dominosee.eventorize` module, which identifies events based on a specified threshold and direction (above or below). For drought events, we're interested in SPI values below a threshold of -1.0. It is recommended to only consider the burst timing for each series of consecutive event occurrences to avoid the effects of temporal clustering on ES calculations, so we set `select="start"` and `select_period=1`.

In [5]:
from dominosee.eventorize import get_event
da_burst = get_event(spi.SPI1, threshold=-1.0, extreme="below", event_name="drought", select="start", select_period=1)
da_burst

<xarray.DataArray 'drought' (lat: 20, lon: 20, time: 365)> Size: 146kB
False False True False False False False ... True False True False False False
Coordinates:
  * lat      (lat) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lon      (lon) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * time     (time) datetime64[ns] 3kB 1950-01-01 1950-01-02 ... 1950-12-31
Attributes:
    threshold:      -1.0
    extreme:        below
    long_name:      drought events
    description:    Events with -1.0 below threshold
    event_name:     drought
    select:         start
    select_period:  1

After identifying extreme events with the `get_event` function, we need to extract the timing positions of these events using `get_event_positions`. This is because ES analysis requires the exact timing of events to calculate the time delay between event pairs between two locations.

In [6]:
from dominosee.es import get_event_positions
da_burst_ep = get_event_positions(da_burst)
da_burst_ep

<xarray.Dataset> Size: 106kB
Dimensions:          (lat: 20, lon: 20, event: 64)
Coordinates:
  * lat              (lat) float64 160B -90.0 -80.53 -71.05 ... 71.05 80.53 90.0
  * lon              (lon) float64 160B -180.0 -161.1 -142.1 ... 161.1 180.0
  * event            (event) int64 512B 0 1 2 3 4 5 6 7 ... 57 58 59 60 61 62 63
Data variables:
    event_positions  (lat, lon, event) int32 102kB 2 12 27 34 37 ... -1 -1 -1 -1
    event_count      (lat, lon) int64 3kB 41 42 48 44 51 55 ... 43 47 45 46 56

The result is a integer Dataset of event positions, where:
- `event_position` indicates the position of the event in the time dimension
- `event_count` indicates the number of events

This integer representation allows us to easily identify the positions of events in the time dimension, and the number of events. The event Dataset includes attributes that document the threshold and criteria used to define the events.

### Calculate Event Synchronization between location pairs

Event Synchronization quantifies the similarity in the timing of events between two time series. Unlike ECA, which counts coincidences within fixed time windows, ES dynamically adapts the tolerance window based on the local event lags.

In [10]:
from dominosee.es import get_event_sync_from_positions
da_es = get_event_sync_from_positions(positionsA=da_burst_ep.event_positions,
                                      positionsB=da_burst_ep.event_positions,
                                      tm=10)
da_es

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<xarray.DataArray 'event_positions' (latA: 20, latB: 20, lonA: 20, lonB: 20)> Size: 160kB
39 11 9 16 14 18 14 10 14 14 16 11 15 ... 17 16 16 16 14 18 9 15 13 14 18 12 54
Coordinates:
  * latA     (latA) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonA     (lonA) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * latB     (latB) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonB     (lonB) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
Attributes:
    long_name:     Event Synchronization
    units:         count
    description:   Number of synchronized events between locations A and B
    tau_max:       10
    output_dtype:  <class 'numpy.uint8'>

The `get_event_sync_from_positions` function calculates the event synchronization between all pairs of grid points. The resulting DataArray has four dimensions:
- `latA`, `lonA`: Coordinates of the location A
- `latB`, `lonB`: Coordinates of the location B

The function parameters control the ES calculation:
- `tm`: The maximum time window (in time steps) within which events must occur to be considered synchronized

### Calculate the statistical confidence of ES

<div style="background-color: #fff3cd; color: #856404; padding: 10px; border-left: 5px solid #ffeeba; margin-bottom: 15px;">
<strong>EXPERIMENTAL:</strong> The Event Synchronization null model in dominosee is currently experimental. The functionality may change in future releases.
</div>

Event Synchronization (ES) is a statistical method used to quantify the synchrony between event time series...

After calculating the raw event coincidence counts, the next critical step is to determine which connections are statistically significant. This involves:

1. **Get critical values**: Computing ES values for many realizations of the surrogate data. Unlike ECA, which can utilize analytical formulas for significance testing, ES typically relies on computational methods like bootstrapping to generate surrogate data. The significance level (e.g., p < 0.05 or p < 0.01) determines how conservative our network construction will be - stricter thresholds produce sparser but more reliable networks.

2. **Construct a null model for location pairs**: Map the critical values to the number of total events in the locations pairs.

In [28]:
from dominosee.es import create_null_model_from_indices
da_cvalues = create_null_model_from_indices(spi.time, tm=da_es.attrs["tau_max"],
                                            max_events=da_burst_ep.event_count.max().values, significances=0.001)
da_cvalues

<xarray.DataArray (noeA: 65, noeB: 65, significance: 1)> Size: 34kB
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 15 14 16 17 16 15 15 18 17 18 17 17 17 18 17
Coordinates:
  * noeA          (noeA) int64 520B 0 1 2 3 4 5 6 7 ... 57 58 59 60 61 62 63 64
  * noeB          (noeB) int64 520B 0 1 2 3 4 5 6 7 ... 57 58 59 60 61 62 63 64
  * significance  (significance) float64 8B 0.001
Attributes:
    description:  Event synchronization null model for pairs of number of events
    tau_max:      10
    max_events:   64
    min_es:       None

In [29]:
from dominosee.es import convert_null_model_for_locations
da_null = convert_null_model_for_locations(da_cvalues, da_burst_ep.event_count, da_burst_ep.event_count, sig=0.001)
da_null

<xarray.DataArray (latA: 20, lonA: 20, latB: 20, lonB: 20)> Size: 1MB
12 10 11 10 12 12 12 11 12 12 11 9 12 ... 13 14 11 14 13 11 14 14 12 14 13 13 15
Coordinates:
    noeA          (latA, lonA) int64 3kB 41 42 48 44 51 55 ... 51 43 47 45 46 56
    noeB          (latB, lonB) int64 3kB 41 42 48 44 51 55 ... 51 43 47 45 46 56
    significance  float64 8B 0.001
  * latA          (latA) float64 160B -90.0 -80.53 -71.05 ... 71.05 80.53 90.0
  * lonA          (lonA) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * latB          (latB) float64 160B -90.0 -80.53 -71.05 ... 71.05 80.53 90.0
  * lonB          (lonB) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
Attributes:
    description:  Event synchronization null model for pairs of locations
    tau_max:      10
    max_events:   64
    min_es:       None

### Construct network represented by adjacency matrix

The next step involves constructing the climate network based on statistically significant connections. We'll create an adjacency matrix, which is a fundamental representation of the network structure:

- **Adjacency Matrix**: A square matrix where both rows and columns correspond to grid points (locations).
- **Matrix Elements**: Each element (i,j) contains a binary value:
  - 1 if a statistically significant connection exists between locations i and j
  - 0 if no significant connection is present
- **Interpretation**: This matrix provides a concise representation of the network topology, enabling further analysis of the climate system's spatial relationships.

In [30]:
from dominosee.network import get_link_from_critical_values

da_link = get_link_from_critical_values(da_es, da_null, rule="greater")
da_link

<xarray.DataArray (latA: 20, latB: 20, lonA: 20, lonB: 20)> Size: 160kB
True True False True True True True ... False True True False True False True
Coordinates:
  * latA          (latA) float64 160B -90.0 -80.53 -71.05 ... 71.05 80.53 90.0
  * lonA          (lonA) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * latB          (latB) float64 160B -90.0 -80.53 -71.05 ... 71.05 80.53 90.0
  * lonB          (lonB) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
    noeA          (latA, lonA) int64 3kB 41 42 48 44 51 55 ... 51 43 47 45 46 56
    noeB          (latB, lonB) int64 3kB 41 42 48 44 51 55 ... 51 43 47 45 46 56
    significance  float64 8B 0.001

In [31]:
print(f"The network density is {da_link.sum().values/da_link.size*100:.2f}%.")

The network density is 82.36%.
